In [185]:
import utils
import numpy as np
import pandas as pd

In [186]:
def is_in_notebook():
    import sys
    return 'ipykernel' in sys.modules
def clear_output():
    """
    clear output for both jupyter notebook and the console
    """
    import os
    os.system('cls' if os.name == 'nt' else 'clear')
    if is_in_notebook():
        from IPython.display import clear_output as clear
        clear()

In [187]:
data_select = utils.load_sql_file("./sql/premium_h5_pk_decision_logic_data_selection.sql")
data_select

'select\n    t1.*,\n    t2.*\nfrom\n    (\n        select\n            order_id,\n            bubble_trace_id,\n            concat_ws(\',\', collect_set(product_category)) as product_set,\n            count(distinct product_category) as pl_num\n        from\n            (\n                select\n                    order_id,\n                    bubble_trace_id,\n                    case\n                        when sub_product_line = 9 then \'Luxury\'\n                        when sub_product_line in (1, 6)\n                        and require_level in (100) then \'Premium\'\n                        when sub_product_line in (1, 6)\n                        and require_level in (400) then \'CommercialPremium\'\n                        when sub_product_line in(3, 7, 314)\n                        and require_level in (600)\n                        and combo_type in (4)\n                        and carpool_type in (1, 2) then \'Carpool\'\n                        when sub_product_line in 

In [188]:
data_select = spark.sql(data_select)
data_select.cache()
data_select.head(5)

[Row(order_id=35233258186349, bubble_trace_id=u'0003ff985fcb9c2f000052546db20735', product_set=u'Other,Flash,SpFlash', pl_num=3, traceId=u'0003ff985fcb9c2f000052546db20735', athena_id=None, passenger_id=u'17592478226663', premiumH5AcceptRatio=u'0.144905', response_rate=u'0.746751', pre_price=u'2300', AcceptRateThreshold=u'0.155851', PriceRatio=u'2', AnswerRateGap=u'-0.1', final_fee=u'1520', model_result=u'{"accept_rate":0.9128656387329102,"answer_rate":0.423454}', accept_rate=u'0.9128656387329102', answer_rate=u'0.423454', product_athena_key=u'SpFlash', dt=u'2020-12-05', hour=u'22'),
 Row(order_id=35233258186349, bubble_trace_id=u'0003ff985fcb9c2f000052546db20735', product_set=u'Other,Flash,SpFlash', pl_num=3, traceId=u'0003ff985fcb9c2f000052546db20735', athena_id=None, passenger_id=u'17592478226663', premiumH5AcceptRatio=u'0.144905', response_rate=u'0.746751', pre_price=u'2300', AcceptRateThreshold=u'0.155851', PriceRatio=u'2', AnswerRateGap=u'-0.1', final_fee=u'1650', model_result=u'

In [189]:
data_select.columns

['order_id',
 'bubble_trace_id',
 'product_set',
 'pl_num',
 'traceId',
 'athena_id',
 'passenger_id',
 'premiumH5AcceptRatio',
 'response_rate',
 'pre_price',
 'AcceptRateThreshold',
 'PriceRatio',
 'AnswerRateGap',
 'final_fee',
 'model_result',
 'accept_rate',
 'answer_rate',
 'product_athena_key',
 'dt',
 'hour']

In [190]:
x = data_select.head(5)
x

[Row(order_id=35233258186349, bubble_trace_id=u'0003ff985fcb9c2f000052546db20735', product_set=u'Other,Flash,SpFlash', pl_num=3, traceId=u'0003ff985fcb9c2f000052546db20735', athena_id=None, passenger_id=u'17592478226663', premiumH5AcceptRatio=u'0.144905', response_rate=u'0.746751', pre_price=u'2300', AcceptRateThreshold=u'0.155851', PriceRatio=u'2', AnswerRateGap=u'-0.1', final_fee=u'1520', model_result=u'{"accept_rate":0.9128656387329102,"answer_rate":0.423454}', accept_rate=u'0.9128656387329102', answer_rate=u'0.423454', product_athena_key=u'SpFlash', dt=u'2020-12-05', hour=u'22'),
 Row(order_id=35233258186349, bubble_trace_id=u'0003ff985fcb9c2f000052546db20735', product_set=u'Other,Flash,SpFlash', pl_num=3, traceId=u'0003ff985fcb9c2f000052546db20735', athena_id=None, passenger_id=u'17592478226663', premiumH5AcceptRatio=u'0.144905', response_rate=u'0.746751', pre_price=u'2300', AcceptRateThreshold=u'0.155851', PriceRatio=u'2', AnswerRateGap=u'-0.1', final_fee=u'1650', model_result=u'

# pk决策时，因为不同勾选业务线过滤冒泡数与过滤比

In [197]:
def map_func(x):
    k, vv = x
    trace_id = k
    rst = dict()
    multi_unavailable_list = set()
    answer_multi_unavailable_list = set()
    price_multi_unavailable_list = set()
    accept_multi_unavailable_list = set()
    single_available_list = set()
    pl_num = 0
    product_set = []
    '''
    Build multi_unavailable_list and single_available_list
    '''
    for v in vv:
        product_set = v["product_set"].split(",")
        pl_num = int(v["pl_num"])
        premiumH5AcceptRatio = float(v["premiumH5AcceptRatio"])
        response_rate = float(v["response_rate"])
        pre_price = float(v["pre_price"])
        AcceptRateThreshold = 0.155851#float(v["AcceptRateThreshold"])
        PriceRatio = 2#float(v["PriceRatio"])
        AnswerRateGap = -0.1#float(v["AnswerRateGap"])
        final_fee = float(v["final_fee"])
        accept_rate = float(v["accept_rate"])
        answer_rate = float(v["answer_rate"])
        product_athena_key = v["product_athena_key"]
        product_judge_list = ["Carpool", "SpFlash", "APlusFlash", "Unione", "YouXiang"]
        total_judge = False
        if product_athena_key in product_judge_list:
            #Add Filter Reason
            if response_rate < answer_rate + AnswerRateGap:
                answer_multi_unavailable_list.add(product_athena_key)
            if pre_price > final_fee + PriceRatio:
                price_multi_unavailable_list.add(product_athena_key)
            if premiumH5AcceptRatio <= AcceptRateThreshold:
                accept_multi_unavailable_list.add(product_athena_key)
            #Add multi_unavailavailable_list and single_available_list
            if response_rate < answer_rate + AnswerRateGap \
            or pre_price > final_fee + PriceRatio \
            or premiumH5AcceptRatio <= AcceptRateThreshold:
                multi_unavailable_list.add(product_athena_key)
            else:
                single_available_list.add(product_athena_key)
        else:
            if product_athena_key != "Flash":
                multi_unavailable_list.add(product_athena_key)
        #Add Flash Product In the End
        single_available_list.add("Flash")
    '''
    PK Decision
    '''
    guide_judge = False
    if pl_num == 1:
        inner_set = single_available_list.intersection(product_set)
        if len(inner_set) != 0:
            guide_judge = True
        else:
            for product in product_set:#遍历勾选集合
                total_filter, answer_filter, price_filter, accept_filter, other_filter = 1, 0, 0, 0, 0
                judge = True
                if product in answer_multi_unavailable_list:
                    judge = False
                    answer_filter = 1
                if product in price_multi_unavailable_list:
                    judge = False
                    price_filter = 1
                if product in accept_multi_unavailable_list:
                    judge = False
                    accept_filter = 1
                if judge and product in multi_unavailable_list:
                    other_filter = 1
                rst[product] = [total_filter, answer_filter, price_filter, accept_filter, other_filter]
                
    else:
        inner_set = multi_unavailable_list.intersection(product_set)
        if len(inner_set) == 0:
            guide_judge = True
        else:
            for product in inner_set:#遍历勾选集合与多发不可导集合交集
                total_filter, answer_filter, price_filter, accept_filter, other_filter = 1, 0, 0, 0, 0
                judge = True
                if product in answer_multi_unavailable_list:
                    judge = False
                    answer_filter = 1
                if product in price_multi_unavailable_list:
                    judge = False
                    price_filter = 1
                if product in accept_multi_unavailable_list:
                    judge = False
                    accept_filter = 1
                if judge:
                    other_filter = 1
                rst[product] = [total_filter, answer_filter, price_filter, accept_filter, other_filter]
    result = []
    for key in rst:
        tmp = (key, rst[key])
        result.append(tmp)
    return result

In [198]:
map_func(("000081e15fc5df7b000054ea84acacfc", x))

[(u'SpFlash', [1, 0, 1, 1, 0])]

In [199]:
def reduce_func(v1, v2):
    return list(map(sum, zip(v1, v2))) 

In [200]:
rst = data_select.rdd \
    .map(lambda x: (x["bubble_trace_id"], x)) \
    .groupByKey() \
    .flatMap(map_func) \
    .reduceByKey(reduce_func) \
    .collect()
rst

[(u'Unione', [163871, 80760, 155121, 105403, 0]),
 (u'SpFlash', [283854, 39631, 227528, 163503, 0]),
 (u'APlusFlash', [107291, 63413, 95988, 60289, 0]),
 (u'Carpool', [36330, 16750, 36327, 29058, 0]),
 (u'CommercialPremium', [5595, 0, 0, 0, 5595]),
 (u'Premium', [54796, 0, 0, 0, 54796]),
 (u'Luxury', [977, 0, 0, 0, 977]),
 (u'Other', [4508, 0, 0, 0, 0]),
 (u'YouXiang', [64787, 38143, 50836, 23185, 0])]

In [201]:
def to_pandas(data):
    for k,v in data:
        v.append(k)
    rst = pd.DataFrame([v for _, v in data])
    rst.columns = [
        "total_filter",
        "answer_filter",
        "price_filter",
        "accept_filter",
        "other_filter",
        "product_line"
    ]
    return rst

In [202]:
rst_pd = to_pandas(rst)
rst_pd["answer_filter_rate"] = rst_pd["answer_filter"] / rst_pd["total_filter"]
rst_pd["price_filter_rate"] = rst_pd["price_filter"] / rst_pd["total_filter"]
rst_pd["accept_filter_rate"] = rst_pd["accept_filter"] / rst_pd["total_filter"]
rst_pd["other_filter_rate"] = rst_pd["other_filter"] / rst_pd["total_filter"]
rst_pd

,total_filter,answer_filter,price_filter,accept_filter,other_filter,product_line,answer_filter_rate,price_filter_rate,accept_filter_rate,other_filter_rate
0,163871,80760,155121,105403,0,Unione,0.492827,0.946604,0.643207,0
1,283854,39631,227528,163503,0,SpFlash,0.139618,0.801567,0.576011,0
2,107291,63413,95988,60289,0,APlusFlash,0.591037,0.894651,0.561920,0
3,36330,16750,36327,29058,0,Carpool,0.461051,0.999917,0.799835,0
4,5595,0,0,0,5595,CommercialPremium,0.000000,0.000000,0.000000,1
5,54796,0,0,0,54796,Premium,0.000000,0.000000,0.000000,1
6,977,0,0,0,977,Luxury,0.000000,0.000000,0.000000,1
7,4508,0,0,0,0,Other,0.000000,0.000000,0.000000,0
8,64787,38143,50836,23185,0,YouXiang,0.588745,0.784664,0.357865,0


In [203]:
rst_pd.to_csv("./rst/filter_reason_rate.csv", index=False, header=True, sep=",", encoding="utf-8_sig")

In [41]:
a = np.array([1,2,3])
b = np.array([4,5,6])
list(a + b)

[5, 7, 9]

In [39]:
np.sum([a,b])

21

In [94]:
a = np.array([1,2])
b = np.array([5,6])
list(a + b)

[6, 8]

In [6]:
",".join([str(x) for x in [1.2,2.5,3.2]])

'1.2,2.5,3.2'

In [7]:
def reduce_func(v1, v2):
    return list(map(sum, zip(v1, v2))) 

In [15]:
v1 = 2.#[1865.0,2.]
v2 = 3.#[1800.0, 3]

In [16]:
reduce_func(v1, v2)

TypeError: zip argument #1 must support iteration